# Test fl_core.py

In [1]:
import numpy as np
import fl_core

w_ofNid1 = [ np.array([[4, 0], [4, 0]], dtype=np.float32), np.array([8, 0, 8], dtype=np.float32) ]
w_ofNid2 = [ np.array([[0, 4], [0, 4]], dtype=np.float32), np.array([0, 8, 0], dtype=np.float32) ]
w_byTime_byNid = [ [w_ofNid1], [w_ofNid2] ]
weight_byNid = [ 1, 3 ]
w_byTime = fl_core.federated_aggregate(w_byTime_byNid, weight_byNid)

assert( w_byTime[0][0].tolist() == [[1, 3], [1, 3]] )
assert( w_byTime[0][1].tolist() == [2, 6, 2] )
assert( type(w_byTime[0][0][0,0]) == np.float32 )
assert( np.linalg.norm(fl_core.np_flatten(w_ofNid1) - fl_core.np_flatten(w_ofNid2)) == 16 )
print('Completed')

W1015 17:15:23.924251 140691690104576 deprecation_wrapper.py:119] From /home/dmlab/prosopher/FL-Sim/fl_core.py:6: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



Completed


# Test fl_alg.py